In [6]:
import numpy as np
import pandas as pd
import requests
import config
import os
import json
from pandas.io.json import json_normalize
import pygsheets

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_

In [183]:
### MULTI-CANDIDATE SEARCH ###
#Set search for all 2020 (two-year transaction period) Maine Congressional candidates
cand_name = ['COLLINS, SUSAN','MCCONNELL, MITCH']
period='2020'

#Initialize ID
id=int(0)
parameters = {'election_year':period
            ,'q':cand_name[id]
            ,'api_key':config.api_key}

#Initialize comm_ids
cand_list=[]
r_cands=[]

for x in range(2):
    
    parameters.update(q=cand_name[id])
    #Requests candidate info
    r_cands = requests.get('https://api.open.fec.gov/v1/candidates/search',params=parameters).json()
    
    #Locates and sets Committee ID from 'principal_committees' sub-array
    cand_list.append(json_normalize(data=r_cands['results'],record_path='principal_committees')[['committee_id','name','party']])
    id=id+1
    
#Print list to validate
cand_list

[  committee_id                 name party
 0    C00314575  COLLINS FOR SENATOR   REP,
   committee_id                        name party
 0    C00193342  MCCONNELL SENATE COMMITTEE   REP
 1    C00155051  MCCONNELL SENATE COMMITTEE   REP]

In [184]:
#C00155051 refers to old McConnell campaign committee
#Remove that item and reform list

#Collins & McConnell joined
cand_ids = [cand_list[0]['committee_id'],cand_list[1][0:1]['committee_id']]
cand_ids

[0    C00314575
 Name: committee_id, dtype: object, 0    C00193342
 Name: committee_id, dtype: object]

In [190]:
#Initialize dataframe collector for itemized contribs
dfs=[]
id=int(0)
p=int(1)
    
#Loop through pages
for x in range(len(cand_ids)):
    
    #Print out candidate
    print(cand_list[id]['name'])
    
    querydict = {'per_page':'100'
                ,'api_key':config.api_key
                ,'committee_id':cand_ids[id]
                ,'page':p
                ,'sort_nulls_last':'false'
                ,'sort':'-contribution_receipt_date'
                ,'sort_hide_null':'false'
                }
    
    r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/efile/',params=querydict).json()
    
    #Loop through pages
    while p <= r['pagination']['pages']:
        
        print('page: '+str(p)+','+str(r['pagination']['per_page'])+' records')
        
        querydict.update(page=p)
        #Pull new results
        r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/efile/',params=querydict).json()
        
        #Add results to dataframe
        df = json_normalize(r['results'])
        dfs.append(df)
        p=p+1
    
    #Reset to page 1
    p=int(1)
    #Increment to next candidate
    id=id+1

print('Job Complete')

0    COLLINS FOR SENATOR
Name: name, dtype: object
page: 1,100 records
page: 2,100 records
page: 3,100 records
page: 4,100 records
page: 5,100 records
page: 6,100 records
page: 7,100 records
page: 8,100 records
page: 9,100 records
page: 10,100 records
page: 11,100 records
page: 12,100 records
page: 13,100 records
page: 14,100 records
page: 15,100 records
page: 16,100 records
page: 17,100 records
page: 18,100 records
page: 19,100 records
page: 20,100 records
page: 21,100 records
page: 22,100 records
page: 23,100 records
page: 24,100 records
page: 25,100 records
page: 26,100 records
page: 27,100 records
page: 28,100 records
page: 29,100 records
page: 30,100 records
page: 31,100 records
page: 32,100 records
page: 33,100 records
page: 34,100 records
page: 35,100 records
page: 36,100 records
page: 37,100 records
page: 38,100 records
page: 39,100 records
page: 40,100 records
page: 41,100 records
page: 42,100 records
page: 43,100 records
page: 44,100 records
0    MCCONNELL SENATE COMMITTEE
1 

In [195]:
itemall=pd.concat(dfs,sort=False)
itemall=itemall.drop_duplicates(subset='transaction_id')
itemdf

,committee.city,committee.committee_id,committee.committee_type_full,committee.cycle,committee.name,committee.party_full,committee.state_full,contribution_receipt_amount,contribution_receipt_date,contributor_aggregate_ytd,...,entity_type,filing.coverage_end_date,filing.coverage_start_date,filing.filed_date,filing.is_amended,line_number,fec_election_type_desc,memo_text,filing.pdf_url,transaction_id
0,BANGOR,C00314575,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,2800.00,2019-06-30,5600.00,...,IND,2019-06-30,2019-04-01,2019-07-15,False,12,PRIMARY,None,http://docquery.fec.gov/pdf/841/20190715915115...,A60F95D3FAE2B4FD0909
1,BANGOR,C00314575,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,250.00,2019-06-30,250.00,...,IND,2019-06-30,2019-04-01,2019-07-15,False,12,PRIMARY,None,http://docquery.fec.gov/pdf/841/20190715915115...,A7563EB6CEE424EE6B90
2,BANGOR,C00314575,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,2800.00,2019-06-30,2800.00,...,IND,2019-06-30,2019-04-01,2019-07-15,False,12,PRIMARY,None,http://docquery.fec.gov/pdf/841/20190715915115...,A9273561CA7524445A8C
3,BANGOR,C00314575,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,2000.00,2019-06-30,2000.00,...,IND,2019-06-30,2019-04-01,2019-07-15,False,12,PRIMARY,None,http://docquery.fec.gov/pdf/841/20190715915115...,A9CE5AABBC81C441A827
4,BANGOR,C00314575,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,200.00,2019-06-30,400.00,...,IND,2019-06-30,2019-04-01,2019-07-15,False,12,PRIMARY,None,http://docquery.fec.gov/pdf/841/20190715915115...,A5376D998DD9D445F8EE
5,BANGOR,C00314575,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,750.00,2019-06-30,750.00,...,IND,2019-06-30,2019-04-01,2019-07-15,False,12,PRIMARY,None,http://docquery.fec.gov/pdf/841/20190715915115...,AD0F4D9FD7A6A461E839
6,BANGOR,C00314575,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,2800.00,2019-06-30,2800.00,...,IND,2019-06-30,2019-04-01,2019-07-15,False,12,PRIMARY,None,http://docquery.fec.gov/pdf/841/20190715915115...,AB086963DB6C2407AAA0
7,BANGOR,C00314575,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,1000.00,2019-06-30,3600.00,...,IND,2019-06-30,2019-04-01,2019-07-15,False,12,PRIMARY,None,http://docquery.fec.gov/pdf/841/20190715915115...,A833561F6709D4AF0906
8,BANGOR,C00314575,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,1000.00,2019-06-30,1000.00,...,IND,2019-06-30,2019-04-01,2019-07-15,False,12,PRIMARY,None,http://docquery.fec.gov/pdf/841/20190715915115...,A62DBED096B1E4786B68
9,BANGOR,C00314575,Senate,2020,COLLINS FOR SENATOR,REPUBLICAN PARTY,Maine,1000.00,2019-06-30,1000.00,...,IND,2019-06-30,2019-04-01,2019-07-15,False,12,PRIMARY,None,http://docquery.fec.gov/pdf/841/20190715915115...,A88B56F52279840EEB07


In [196]:
#Itemized dataframe query
itemdf=itemdf[[
                'committee.city'
                ,'committee.committee_id'
                ,'committee.committee_type_full'
                ,'committee.cycle'
                ,'committee.name'
                ,'committee.party_full'
                ,'committee.state_full'
                ,'contribution_receipt_amount'
                ,'contribution_receipt_date'
                ,'contributor_aggregate_ytd'
                ,'contributor_city'
                ,'contributor_employer'
                ,'contributor_first_name'
                ,'contributor_last_name'
                ,'contributor_name'
                ,'cycle'
                ,'contributor_occupation'
                ,'contributor_state'
                ,'contributor_zip'
                ,'entity_type'
                ,'filing.coverage_end_date'
                ,'filing.coverage_start_date'
                ,'filing.filed_date'
                ,'filing.is_amended'
                ,'line_number'
                ,'fec_election_type_desc'
                ,'memo_text'
                ,'filing.pdf_url'
                ,'transaction_id']]
len(itemdf.index)

8430

In [193]:
#Write itemized individual results to local CSV
cwd = os.getcwd()
itemdf.to_csv(cwd+'/data/collins-and-mitch-schedule-a.csv')

In [ ]:
# #Write itemized individual raw results to Google Sheet
# cwd = os.getcwd()

# #Google Credentials
# gc = pygsheets.authorize(service_file=cwd+'/me-congress-2020-creds.json')

# #Select sheet and worksheet
# sh = gc.open('me-congress-2020')
# #sh = gc.open_by_key('1AKrgHT9NLpoddV16B7_M_0PEjJmMQAGtXJUnLCTDHjA')
# wks = sh[3]

# #Clear sheet before load
# wks.clear(start='A1',fields='*')

# #Write contribs dataframe to sheet
# wks.set_dataframe(itemdf,(1,1))